In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from transformers import VisionEncoderDecoderModel, DonutProcessor
from PIL import Image
import re

def donut_inference(image_path,model_path ,base_model="naver-clova-ix/donut-base-finetuned-docvqa"):
    # Load the processor and model
    processor = DonutProcessor.from_pretrained(base_model)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = torch.load(model_path,map_location=torch.device(device))
    model.to(device)



    # Load and prepare the image
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    # Prepare the decoder input
    decoder_input_ids = torch.full((1, 1), model.config.decoder_start_token_id, device=device)

    # Perform inference
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=model.decoder.config.max_position_embeddings,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
        output_scores=True,
    )

    # Decode and clean up the output
    seq = processor.batch_decode(outputs.sequences)[0]
    seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    seq = re.sub(r"<.*?>", "", seq, count=1).strip()

    # Convert the sequence to JSON if applicable
    json_output = processor.token2json(seq)

    return json_output



<ipython-input-10-4602a6cde46c>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path,map_location=torch.device(device))
/usr/local/lib/python3.10/

{'text_sequence': 'passport'}


In [ ]:
image_path = '/content/Bangladeshi_Passport_Bio_Page.jpg'
res = donut_inference(image_path,'/content/drive/MyDrive/donutData/finalModelPtFull/model_final_0.pth')
print(res)